In [2]:

#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()
import tensorflow as tf
# from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow.python import keras
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from os.path import isfile
import os
# import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import time
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split
import numpy as np
import shutil

# tensorflow v1版的import
# from tensorflow import keras
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from keras.callbacks import ModelCheckpoint





ModuleNotFoundError: No module named 'sklearn'

In [2]:
def build_model():
    #     model = tf.keras.models.Sequential([
    #       tf.keras.layers.Flatten(input_shape=(28, 28)),
    #       tf.keras.layers.Dense(128, activation='relu'),
    #       tf.keras.layers.Dropout(0.2),
    #       tf.keras.layers.Dense(10, activation='softmax')
    #     ])
    model = Sequential() # create model
    model.add(Dense(5, input_dim=8, activation='relu')) # hidden layer
    model.add(Dense(1, activation='sigmoid'))      
    return model



In [3]:
def save_model_to_serving(model, export_version, export_path='prod_models'):
    # 只能用v1版本的tensorflow
    import tensorflow.compat.v1 as tf
    tf.disable_v2_behavior()
    print(model.input, model.output)
    signature = tf.saved_model.signature_def_utils.predict_signature_def(                                                                        
        inputs={'voice': model.input}, outputs={'scores': model.output})
    export_path = os.path.join(
        tf.compat.as_bytes(export_path),
        tf.compat.as_bytes(str(export_version)))
    builder = tf.saved_model.builder.SavedModelBuilder(export_path)
    legacy_init_op = tf.group(tf.tables_initializer(), name='legacy_init_op')
    # tensorflow v1,v2区别在于v2集成了keras，
    builder.add_meta_graph_and_variables(
        sess=K.get_session(),                                                                                                                    
        tags=[tf.saved_model.tag_constants.SERVING],                                                                                             
        signature_def_map={                                                                                                                      
            'voice_classification': signature,                                                                                                                     
        })  #
    builder.save()


## 第一步，定义并训练模型并保存checkpoint

In [4]:
#使用numpy生成假数据。共dataset_size个学习样本
dataset_size = 4230
x_ =np.random.rand(dataset_size,8)
# y = 2*x1 + 7*x2 + 0.3
u = map(lambda x1, x2, x3, x4, x5, x6, x7, x8: 2.0*x1 + 7.0*x2 + x3*0.45 + x4*36 + x5*6 + x6*120 + x7*4 + x8*3.5
        + 0.3, x_[:, 0], x_[:, 1], x_[:, 2], x_[:, 3], x_[:, 4], x_[:, 5], x_[:, 6], x_[:, 7])
y_ = np.array([[i] for i in u])

# fix random seed for reproducibility
seed = 155
np.random.seed(seed)
X_train, X_test, Y_train, Y_test = train_test_split(x_, y_,
                                                    test_size=0.25, random_state=87)
np.random.seed(seed)
my_first_nn = Sequential() # create model
my_first_nn.add(Dense(5, input_dim=8, activation='relu')) # hidden layer
my_first_nn.add(Dense(1, activation='sigmoid'))           # output layer
my_first_nn.compile(loss='mse', optimizer='adam', metrics=['mape'])

# optional

# specify filepath- this will write a new file for each epoch with the epoch number contained within the filename
filename = "nn_weights-{epoch:02d}.hdf5"
filepath = Path().resolve()/ "checkpoint" / filename
if os.path.exists(str(filepath.parent)):
    #     print(str(filepath.parent))
    #os.rmdir(str(filepath.parent))  # 只能删除空文件夹
    shutil.rmtree(str(filepath.parent),True) 
    os.mkdir(str(filepath.parent))
else:
    os.mkdir(str(filepath.parent))
print(filepath)
checkpoint = keras.callbacks.ModelCheckpoint(str(filepath), monitor='val_mape', verbose=0,
                                             save_weights_only=False, save_best_only=False, mode='min')

# verbose=0 suppresses the file writing message
# note that the fit method expects a list of callbacks
my_first_nn_fitted = my_first_nn.fit(X_train, Y_train, epochs=100, verbose=0, batch_size=32,
                                     callbacks=[checkpoint], initial_epoch=0)

# [loss, accuracy]
my_first_nn.evaluate(X_test, Y_test, verbose=0)
# training loss and accuracy over the first five epochs
[my_first_nn_fitted.history['loss'][0:5], my_first_nn_fitted.history['mape'][0:5]]




/mnt/python_code/tensorflow/keras/keras_tensorflow_serving/checkpoint/nn_weights-{epoch:02d}.hdf5


[[9228.449955174181,
  9197.346464178752,
  9171.464119640605,
  9157.033575031526,
  9149.73608829455],
 [99.29937, 99.04883, 98.851295, 98.74097, 98.68385]]

In [5]:
# You may also want to visualise how the model performed
# on the test set over time. Remember that we saved the model weights
# to a file at each epoch. This means we can reproduce each intermediate model
# by loading the weights into an appropriately constructed
# (i.e. same number of layers/neurons) neural network.
temp_test_model = Sequential() # create model
temp_test_model.add(Dense(5, input_dim=8, activation='relu')) # hidden layer
temp_test_model.add(Dense(1, activation='sigmoid')) # output layer
temp_test_model.compile(loss='mse', optimizer='adam', metrics=['mape'])
test_over_time = []
# print(my_first_nn_fitted.history.keys())
for i in range(1,len(my_first_nn_fitted.history['loss'])):
    temp_test_model.load_weights(str(Path().resolve()/ "checkpoint" / "nn_weights-%02d.hdf5") % i)
    scores = temp_test_model.evaluate(X_test, Y_test, verbose=0)
    # 0 is loss; 1 is mape
    print("loss:{},mape:{}".format(scores[0],scores[1]))
    test_over_time.append(scores)

loss:9225.753902557892,mape:99.14933013916016
loss:9194.780341741493,mape:98.90576934814453
loss:9174.962120820534,mape:98.75141143798828
loss:9164.916641437263,mape:98.67112731933594
loss:9159.803304805648,mape:98.62895965576172
loss:9157.012649161154,mape:98.60525512695312
loss:9155.358987328686,mape:98.59080505371094
loss:9154.306452327504,mape:98.58137512207031
loss:9153.604095285917,mape:98.574951171875
loss:9153.109699905483,mape:98.5703353881836
loss:9152.749623405009,mape:98.56690216064453
loss:9152.483555352079,mape:98.5643539428711
loss:9152.280773718101,mape:98.5623550415039
loss:9152.121195282964,mape:98.56076049804688
loss:9151.994192314509,mape:98.55946350097656
loss:9151.891437263705,mape:98.55842590332031
loss:9151.806324949788,mape:98.55755615234375
loss:9151.736128751181,mape:98.55680847167969
loss:9151.677664224953,mape:98.55619049072266
loss:9151.627141422496,mape:98.55567169189453
loss:9151.584536345108,mape:98.55519104003906
loss:9151.548126624528,mape:98.55480194

## 第二步，重新用tensorflow定义模型，并将keras训练的权重填充新模型的权重，将新模型保存成tensorflow serving可以识别的格式

In [6]:
model = build_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

checkpoint_filepath = './checkpoint/nn_weights-999.hdf5'
if (isfile(checkpoint_filepath)):
    print('Checkpoint file detected. Loading weights.')
    model.load_weights(checkpoint_filepath) # 加载模型
else:
    print('No checkpoint file detected.  Starting from scratch.')

export_path = "test_model"
save_model_to_serving(model, "1", export_path)
print("save done!")


W1030 15:53:32.362629 140154643892032 deprecation.py:323] From /root/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/compat/v2_compat.py:65: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term
W1030 15:53:32.366569 140154643892032 deprecation.py:323] From /root/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/saved_model/signature_def_utils_impl.py:201: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 6         
Total params: 51
Trainable params: 51
Non-trainable params: 0
_________________________________________________________________
No checkpoint file detected.  Starting from scratch.
Tensor("dense_4_input:0", shape=(?, 8), dtype=float32) Tensor("dense_5/Identity:0", shape=(?, 1), dtype=float32)
save done!
